In [51]:
import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
from scipy.signal import resample_poly, czt
from fractions import Fraction


In [52]:
def prep(file_location:str, fs_out:int, mono:bool = True, max_den:int = 1000):
    fs_in, x = wavfile.read(file_location)
    if np.issubdtype(x.dtype, np.integer):
        x = x.astype(np.float32) / np.iinfo(x.dtype).max
    else:
        x = x.astype(np.float32)
        m = np.max(np.abs(x)) + 1e-12
        if m > 1:
            x /= m
    if mono and x.ndim == 2:
        x = x.mean(axis=1)
    if fs_in != fs_out:
        if x.ndim == 1:
            frac = Fraction(fs_out, fs_in).limit_denominator(max_den)
            x = resample_poly(x, up=frac.numerator, down=frac.denominator)
        else:
            frac = Fraction(fs_out, fs_in).limit_denominator(max_den)
            x = np.column_stack([
                resample_poly(x[:, c], up=frac.numerator, down=frac.denominator)
                for c in range(x.shape[1])
            ])

    return x.astype(np.float32), fs_out

In [53]:
# ---------- RIR(Impulse Response) 생성 (샘플레이트 16 kHz) ----------
def rir(room_size = list, # 가로 5m, 세로 4m, 높이 3m짜리 직육면체 방을 생성
        noise_source = list,
        mic_pos = list, 
        fs_rir = 16000,# fs=fs_rir : 샘플레이트 16kHz로 시뮬레이션
        absorption=0.2, # absorption=0.2 : 벽면 흡수율(0=완전반사, 1=완전흡수)
        max_order=10, # max_order=10 : 10차 반사까지 계산(크면 더 정확하지만 계산량↑)
        
        ):
    room = pra.ShoeBox(room_size, fs=fs_rir, absorption=absorption, max_order=max_order)  
    room.add_source(noise_source)  # 방 안에 (x=2m, y=1.5m, z=1.5m) 위치에 음원(스피커)을 추가
    room.add_microphone_array(np.c_[mic_pos]) # np.c_[[...]] : pyroomacoustics 마이크 배열 입력형태(3×N)로 변환
    room.compute_rir()  
    # 지정된 음원과 마이크 위치를 기반으로
    # 방의 잔향(임펄스 응답)을 실제 음향학 모델로 계산
    rir = room.rir[0][0].astype(np.float32) 
    # 첫 번째 마이크와 첫 번째 소스 사이의 RIR(임펄스 응답)을 가져옴
    # float32로 변환하여 후처리(컨볼루션)에 사용하기 편하게 함
    return rir